In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = '../../assets/doc/ece4010j_syllabus_spring_2025.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

print(len(docs))
print(docs[0].page_content[0:100])
print(docs[0].metadata)

12
Course Description
ECE4010J Probabilistic Methods
in Engineering
Spring 2025
Hello and welcome to EC
{'source': '../../assets/doc/ece4010j_syllabus_spring_2025.pdf', 'page': 0}


In [6]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vector_store = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vector_store.as_retriever()

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "Briefly introduce what 401 does"})

results

{'input': 'Briefly introduce what 401 does',
 'context': [Document(id='c9350101-92f3-4fb7-b006-34a156452326', metadata={'source': '../../assets/doc/ece4010j_syllabus_spring_2025.pdf', 'page': 2}, page_content='The technical course outcomesdefine a set of minimal skills or items of knowledge that you\nshould feel confident in applying after completing ECE4010J. Naturally, the course will cover\nmuch more than just the outcomes listed below. However, they represent a cross-section of basic\nskills and are a useful guide to how well you have mastered the contents of the course in general.\nA quantitative measurement of these outcomes will be provided byCourse Outcome Quizzes,\nwhich will also contribute to the course grade. You will be also asked to report your subjective\nimpression on attaining the outcomes in the Course Evaluation survey at the end of the term.\nAfter completing ECE4010J, students should be able to:\ni) Apply and interpret Bayes’s theorem correctly in a given context.\